In [27]:
import os
import json
import sys

sys.path.insert(0, "../src/")
from data.utils import PlaylistIterator

In [20]:
files = filter(lambda x: ".json" in x and "chunk" in x, os.listdir("../data/processed"))
files = sorted(files, key=lambda x: int(x[6:].split(".")[0]))
files = list(map(lambda x: os.path.join("../data/processed", x), files))
files = files[:-2]

In [21]:
pi = PlaylistIterator(files)

In [24]:
from tqdm import tqdm

songs = set()

for pl in tqdm(pi, total=50000*18):
    songs.update(pl)

100%|████████████████████████████████| 900000/900000 [00:19<00:00, 46778.60it/s]


In [30]:
song2idx = {s: i for i,s in enumerate(songs)}
with open("../models/sparse/idx2song.json", "w") as f:
    json.dump({v:k for k,v in song2idx.items()}, f)

In [31]:
rows, cols, data = [], [], []
for i, pl in tqdm(enumerate(pi), total=50000*18):
    for song in set(pl):
        rows.append(i)
        cols.append(song2idx[song])
        data.append(1)

  6%|█▊                               | 49656/900000 [00:01<00:27, 31204.52it/s]Exception ignored in: <function tqdm.__del__ at 0x7fddaecc32e0>
Traceback (most recent call last):
  File "/home/mbh/.venvs/base/lib/python3.10/site-packages/tqdm/std.py", line 1162, in __del__
    self.close()
  File "/home/mbh/.venvs/base/lib/python3.10/site-packages/tqdm/std.py", line 1281, in close
    if self.disable:
AttributeError: 'tqdm' object has no attribute 'disable'
100%|████████████████████████████████| 900000/900000 [00:42<00:00, 20934.23it/s]


In [35]:
from scipy.sparse import csr_matrix

R = csr_matrix((data, (rows, cols)), shape=(max(rows)+1, max(cols)+1))

In [37]:
from scipy.sparse import save_npz

save_npz("../models/sparse/r.npz", R)

In [45]:
import numpy as np
k = np.ones((2150618, 10))

In [46]:
%%timeit
R @ k

1.82 s ± 60.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
